# Práctica 2

**Datos**

**Alumno:** Alan Ruiz Mondragón  
**Grupo:** 18

### Parte 1 - Ingeniería de variables de texto

Generar una TAD que nos permita poder identificar al usuario a partir del comportamiento dentro de una conversación, utilizando técnicas de ingeniería de variables para poder captar dicho comportamiento.

#### Librerias

In [68]:
from datetime import datetime
from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger
from whatstk import WhatsAppChat
import emoji
import jellyfish as jf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import regex
import seaborn as sns
import unicodedata

#### Dataset

In [69]:
filepath = '/work/files/workspace/practica_2/data/wa_chat.txt'

#### Transformación del dataset

In [70]:
#Cargamos mediante la libreria de whatstk el dataset
chat = WhatsAppChat.from_source(filepath=filepath, hformat='[%y-%m-%d, %H:%M:%S] %name:')
df = chat.df

#Realizamos conversiones según la estructura requerida dejando la misma nomenclatura
df['v_hora'] = df['date'].dt.hour.astype('string')
df['t_fecha'] = df['date'].dt.date.astype('string')
df['v_usuario'] = df['username']
df['t_texto'] = df['message']
df = df[['t_fecha', 'v_hora', 't_texto', 'v_usuario']]

In [71]:
df

,t_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,‎Los mensajes y las llamadas están cifrados de...,Nayeli
1,2021-01-07,23,Que descanses bbcito :),Nayeli
2,2021-01-08,8,Buenosssss días,Nayeli
3,2021-01-08,8,‎Se eliminó este mensaje.,Nayeli
4,2021-01-08,8,Hola buenos días,Alan
...,...,...,...,...
67531,2023-01-15,0,‎imagen omitida ‎,Alan
67532,2023-01-15,0,‎audio omitido ‎,Alan
67533,2023-01-15,0,‎audio omitido ‎,Alan
67534,2023-01-15,0,‎audio omitido ‎,Nayeli


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67536 entries, 0 to 67535
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   t_fecha    67536 non-null  string
 1   v_hora     67536 non-null  string
 2   t_texto    67536 non-null  string
 3   v_usuario  67536 non-null  string
dtypes: string(4)
memory usage: 2.1 MB


#### Missings

In [73]:
#Modificamos la fecha para obtener el insumo requerido
df['t_fecha'] = pd.to_datetime(df['t_fecha'])

#Renombramos la columna
df.rename(columns = {'t_fecha': 'd_fecha'}, inplace = True)

In [74]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,‎Los mensajes y las llamadas están cifrados de...,Nayeli
1,2021-01-07,23,Que descanses bbcito :),Nayeli
2,2021-01-08,8,Buenosssss días,Nayeli
3,2021-01-08,8,‎Se eliminó este mensaje.,Nayeli
4,2021-01-08,8,Hola buenos días,Alan


In [75]:
#Construimos el dataframe de missings 
missings = pd.DataFrame(df.isnull().sum())
missings = missings.reset_index()
missings.rename(columns = {'index': 'nombre_columna', 0: 'total_missings'}, inplace = True)

In [76]:
missings.head()

,nombre_columna,total_missings
0,d_fecha,0
1,v_hora,0
2,t_texto,0
3,v_usuario,0


#### Multimedia omitido

In [77]:
#Se considera todo lo omitido
df.loc[df['t_texto'].str.contains('omitido'), 't_texto'] = '<Multimedia omitido>'
df.loc[df['t_texto'].str.contains('omitida'), 't_texto'] = '<Multimedia omitido>'

In [78]:
total_multimedia = total_multimedia = df[df['t_texto'] == '<Multimedia omitido>'].value_counts().sum()

In [79]:
total_multimedia

18431

In [80]:
#Eliminamos los registros que continene multimedia omitida
df = df[df['t_texto'] != '<Multimedia omitido>']
df.reset_index(drop=True,inplace=True)
df.shape

(49105, 4)

#### Calidad de datos

Hacemos un revisión visual del dataframe generado

In [81]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,‎Los mensajes y las llamadas están cifrados de...,Nayeli
1,2021-01-07,23,Que descanses bbcito :),Nayeli
2,2021-01-08,8,Buenosssss días,Nayeli
3,2021-01-08,8,‎Se eliminó este mensaje.,Nayeli
4,2021-01-08,8,Hola buenos días,Alan


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49105 entries, 0 to 49104
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   d_fecha    49105 non-null  datetime64[ns]
 1   v_hora     49105 non-null  string        
 2   t_texto    49105 non-null  string        
 3   v_usuario  49105 non-null  string        
dtypes: datetime64[ns](1), string(3)
memory usage: 1.5 MB


##### Duplicados

In [83]:
df.duplicated().sum()

437

In [84]:
#Total de registros duplicados de forma general
duplicados = df[df.duplicated(keep = False)]
duplicados.head()

,d_fecha,v_hora,t_texto,v_usuario
163,2021-01-09,13,Te amo,Alan
188,2021-01-09,13,Te amo,Alan
203,2021-01-09,14,JAJAJA ‎,Nayeli
223,2021-01-09,14,JAJAJA ‎,Nayeli
275,2021-01-09,19,Siii,Nayeli


In [85]:
#Eliminamos los duplicados
df.drop_duplicates(inplace=True) 
df.reset_index(drop=True,inplace=True)

In [86]:
#Hacemos un nuevo conteo 
df.duplicated().sum()

0

##### Completitud

In [87]:
def completitud(df):
    comple = pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace = True)
    comple = comple.rename(columns = {'index': 'columna', 0: 'nulos'})
    comple['completitud'] = (1-comple['nulos']/df.shape[0])*100
    comple = comple.sort_values(by = 'completitud', ascending = True)
    comple.reset_index(drop = True, inplace = True)
    return comple

completitud(df)

,columna,nulos,completitud
0,d_fecha,0,100.0
1,v_hora,0,100.0
2,t_texto,0,100.0
3,v_usuario,0,100.0


#### Limpieza de la variable 'v_usuario'

Creamos funciones de limpieza de texto y de emojis

In [88]:
#Función de limpieza de texto
def clean_text(text, pattern="[^a-zA-Z ]"):
    text=str(text) 
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

#Función de limpieza de emojis
def delemoji(text):
	emoji_pattern = re.compile("["
		u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
	                           "]+", flags=re.UNICODE)
	return(emoji_pattern.sub(r'', text))

In [89]:
df["v_usuario"] = df["v_usuario"].map(clean_text)
df["v_usuario"] = df["v_usuario"].map(delemoji)
df['v_usuario'] = df['v_usuario'].astype('string')

In [90]:
df["v_usuario"].unique()

<StringArray>
['nayeli', 'alan']
Length: 2, dtype: string

In [91]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,‎Los mensajes y las llamadas están cifrados de...,nayeli
1,2021-01-07,23,Que descanses bbcito :),nayeli
2,2021-01-08,8,Buenosssss días,nayeli
3,2021-01-08,8,‎Se eliminó este mensaje.,nayeli
4,2021-01-08,8,Hola buenos días,alan


#### Limpieza de la variable 't_texto'

In [92]:
#Eliminamos otros registros que no dan relevancia
df = df[~df['t_texto'].str.contains('drive', regex=False)] 
df = df[~df['t_texto'].str.contains('Se eliminó este mensaje.')]
df = df[~df['t_texto'].str.contains('Eliminaste este mensaje.')] 
df = df[~df['t_texto'].str.contains('Los mensajes y las llamadas están cifrados de extremo a extremo. Nadie fuera de este chat, ni siquiera WhatsApp, puede leerlos ni escucharlos.')] 
df.reset_index(drop=True,inplace=True)

#Guardamos un dataframe para un posible análisis de sentimientos, esto para tratar de mantener el texto casi sin cambios que permita una adecuada traducción
df2 = df

In [93]:
df2.head()

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,Que descanses bbcito :),nayeli
1,2021-01-08,8,Buenosssss días,nayeli
2,2021-01-08,8,Hola buenos días,alan
3,2021-01-08,8,Como amaneciste? ‎,nayeli
4,2021-01-08,8,Súper bien,alan


In [94]:
#Limpiamos la variable texto
df["t_texto"] = df["t_texto"].map(delemoji)
df["t_texto"] = df["t_texto"].map(clean_text)
df = df[df['t_texto'] != '']
df.reset_index(drop=True,inplace=True)
df['t_texto'] = df['t_texto'].astype('string')

In [95]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,que descanses bbcito,nayeli
1,2021-01-08,8,buenosssss dias,nayeli
2,2021-01-08,8,hola buenos dias,alan
3,2021-01-08,8,como amaneciste,nayeli
4,2021-01-08,8,super bien,alan


In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45487 entries, 0 to 45486
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   d_fecha    45487 non-null  datetime64[ns]
 1   v_hora     45487 non-null  string        
 2   t_texto    45487 non-null  string        
 3   v_usuario  45487 non-null  string        
dtypes: datetime64[ns](1), string(3)
memory usage: 1.4 MB


#### Stop words

In [97]:
import nltk
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words("spanish")

[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [98]:
stop_words = list(map(lambda x: clean_text(x), stop_words))
stop_words += ['ah', 'ahh', 'ahhh', 'grax', 'khe', 'kede']

In [99]:
df["t_texto"] = df["t_texto"].map(lambda text:" ".join([x for x in text.split(" ") if x not in stop_words]))

In [100]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,descanses bbcito,nayeli
1,2021-01-08,8,buenosssss dias,nayeli
2,2021-01-08,8,hola buenos dias,alan
3,2021-01-08,8,amaneciste,nayeli
4,2021-01-08,8,super bien,alan


In [101]:
df.shape

(45487, 4)

#### Hapaxes

In [102]:
corpus=" ".join(df["t_texto"].values)
corpus.split()
fdist=nltk.FreqDist(corpus.split())
fdist
pd.DataFrame(fdist.items()).sort_values(by=[1])

,0,1
14932,dataframe,1
10514,valen,1
10515,exagerar,1
10516,demuestre,1
10517,frutaaaa,1
...,...,...
514,amorcito,1268
43,gracias,1413
8,bien,1451
74,amo,2685


In [103]:
list_hapaxes=fdist.hapaxes()
len(list_hapaxes)
len(corpus.split(" "))
len(list_hapaxes)

7990

In [104]:
df['t_texto']=df['t_texto'].map(lambda text:' '.join([x for x in text.split(' ') if x not in list_hapaxes]))
df = df[df['t_texto'] != '']
df.reset_index(drop=True,inplace=True)

In [105]:
df

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,descanses bbcito,nayeli
1,2021-01-08,8,buenosssss dias,nayeli
2,2021-01-08,8,hola buenos dias,alan
3,2021-01-08,8,amaneciste,nayeli
4,2021-01-08,8,super bien,alan
...,...,...,...,...
42437,2023-01-14,23,extrano verdad,nayeli
42438,2023-01-14,23,ojala pronto amor mientras siguele echando pra...,nayeli
42439,2023-01-14,23,dijiste amo,alan
42440,2023-01-15,0,cierto,alan


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42442 entries, 0 to 42441
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   d_fecha    42442 non-null  datetime64[ns]
 1   v_hora     42442 non-null  string        
 2   t_texto    42442 non-null  object        
 3   v_usuario  42442 non-null  string        
dtypes: datetime64[ns](1), object(1), string(2)
memory usage: 1.3+ MB


#### Tokenización

In [107]:
df["t_texto"]=df["t_texto"].map(lambda x:x.split())

In [108]:
df

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,"[descanses, bbcito]",nayeli
1,2021-01-08,8,"[buenosssss, dias]",nayeli
2,2021-01-08,8,"[hola, buenos, dias]",alan
3,2021-01-08,8,[amaneciste],nayeli
4,2021-01-08,8,"[super, bien]",alan
...,...,...,...,...
42437,2023-01-14,23,"[extrano, verdad]",nayeli
42438,2023-01-14,23,"[ojala, pronto, amor, mientras, siguele, echan...",nayeli
42439,2023-01-14,23,"[dijiste, amo]",alan
42440,2023-01-15,0,[cierto],alan


#### Derivación

In [109]:
from nltk.stem import SnowballStemmer
spanish_stemmer=SnowballStemmer('spanish')

In [110]:
text = df.iloc[0]["t_texto"]

In [111]:
text

['descanses', 'bbcito']

In [112]:
text=" ".join([x for x in text if x not in list_hapaxes])

In [113]:
text

'descanses bbcito'

In [114]:
text=text.split(" ")

In [115]:
text

['descanses', 'bbcito']

In [116]:
spanish_stemmer.stem('pronto')

'pront'

In [117]:
print([spanish_stemmer.stem(word) for word in text])

['descans', 'bbcit']


In [118]:
df["t_texto"]=df["t_texto"].map(lambda text:[spanish_stemmer.stem(word) for word in text])

In [119]:
df

,d_fecha,v_hora,t_texto,v_usuario
0,2021-01-07,23,"[descans, bbcit]",nayeli
1,2021-01-08,8,"[buenosssss, dias]",nayeli
2,2021-01-08,8,"[hol, buen, dias]",alan
3,2021-01-08,8,[amanec],nayeli
4,2021-01-08,8,"[sup, bien]",alan
...,...,...,...,...
42437,2023-01-14,23,"[extran, verd]",nayeli
42438,2023-01-14,23,"[ojal, pront, amor, mientr, siguel, echand, pr...",nayeli
42439,2023-01-14,23,"[dij, amo]",alan
42440,2023-01-15,0,[ciert],alan


#### Creación de variables

##### Longitud

In [120]:
#Agregamos una variable con el conteo de numero de palabras
df['c_palabras'] = df["t_texto"].apply(lambda x: len(x))

In [121]:
#Agregamos una variable con el conteo de letras por sentencia
df['c_letras'] =df["t_texto"].map(lambda x: sum(len(word) for word in x))

In [122]:
#Agregamos la longitud de palabras
df['c_long_palabras'] = df['c_letras'] / df['c_palabras']

In [123]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario,c_palabras,c_letras,c_long_palabras
0,2021-01-07,23,"[descans, bbcit]",nayeli,2,12,6.000000
1,2021-01-08,8,"[buenosssss, dias]",nayeli,2,14,7.000000
2,2021-01-08,8,"[hol, buen, dias]",alan,3,11,3.666667
3,2021-01-08,8,[amanec],nayeli,1,6,6.000000
4,2021-01-08,8,"[sup, bien]",alan,2,7,3.500000


##### TF-IDF vectorizer

Finalmente calcularemos la importancia de las palabras menos comúnes.

In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [125]:
df["t_texto"]=df["t_texto"].map(lambda x:" ".join(x))

In [126]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario,c_palabras,c_letras,c_long_palabras
0,2021-01-07,23,descans bbcit,nayeli,2,12,6.000000
1,2021-01-08,8,buenosssss dias,nayeli,2,14,7.000000
2,2021-01-08,8,hol buen dias,alan,3,11,3.666667
3,2021-01-08,8,amanec,nayeli,1,6,6.000000
4,2021-01-08,8,sup bien,alan,2,7,3.500000


In [127]:
#Generamos un nuestro train y test
X_train,X_test=train_test_split(df,test_size=0.20,random_state=42)

In [128]:
vect = TfidfVectorizer()
vect.fit(X_train["t_texto"])
array_train= vect.transform(X_train["t_texto"])
array_test= vect.transform(X_test["t_texto"])

In [129]:
aux_train=pd.DataFrame(array_train.toarray(),columns=vect.get_feature_names())
aux_test=pd.DataFrame(array_test.toarray(),columns=vect.get_feature_names())

In [130]:
aux_test.head()

,aa,ab,abaj,abandon,abb,aber,abiert,about,abrac,abraham,abramz,abraz,abrazadit,abrazam,abrazandot,abrazart,abrazit,abrazooo,abrazot,abre,abres,abri,abrieron,abril,abrir,abrirt,abro,abrum,abrumart,absolut,abu,abuel,abuelit,aburr,aburriri,aburrirt,abus,aca,acab,academ,...,yap,yass,yastoi,yastoy,yay,yayaay,yayay,ybrygolngg,ye,yei,yeii,yeiii,yeiy,yeiyy,yeiyyyy,yend,yenisell,yep,yes,yess,yesssss,yi,yogurt,yoo,yooo,you,yougurt,your,youtu,youtub,yyyyyy,zaragoz,ziert,ziziert,zmj,zml,zocal,zon,zoom,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [131]:
X_train.head()

,d_fecha,v_hora,t_texto,v_usuario,c_palabras,c_letras,c_long_palabras
15851,2021-06-10,16,agust,nayeli,1,5,5.0
29054,2021-11-03,18,reunion,nayeli,1,7,7.0
23962,2021-09-01,20,hoy acab,nayeli,2,7,3.5
40083,2022-11-13,13,entiend,alan,1,7,7.0
17099,2021-06-22,17,tas modorr,alan,2,9,4.5


In [132]:
X_train=pd.concat([X_train.reset_index(drop=True),aux_train.reset_index(drop=True)],axis=1)
#Se añaden las nuevas columnas al dataset train base
X_test=pd.concat([X_test.reset_index(drop=True),aux_test.reset_index(drop=True)],axis=1)
#Se añaden las nuevas columnas al dataset test base

In [133]:
#Tomamos como ejemplo las primeras 43 columnas para no traer todas las nuevas generadas
X_train = X_train.iloc[:,1:50]

In [134]:
X_train.describe()

,c_palabras,c_letras,c_long_palabras,aa,ab,abaj,abandon,abb,aber,abiert,about,abrac,abraham,abramz,abraz,abrazadit,abrazam,abrazandot,abrazart,abrazit,abrazooo,abrazot,abre,abres,abri,abrieron,abril,abrir,abrirt,abro,abrum,abrumart,absolut,abu,abuel,abuelit,aburr,aburriri,aburrirt,abus,aca,acab,academ,acapulc,acces,accident
count,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.00000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000,33953.000000
mean,2.843224,12.686213,4.484813,0.000024,0.000055,0.000327,0.000019,0.000008,0.000371,0.000156,0.000037,0.000167,0.000066,0.000098,0.002478,0.000092,0.000098,0.000035,0.000432,0.000531,0.000029,0.000227,0.000089,0.000062,0.000087,0.000032,0.000064,0.000163,0.000024,0.000051,0.000103,0.000017,0.000042,0.000384,0.00020,0.000429,0.000187,0.000041,0.000036,0.000052,0.001052,0.004168,0.000039,0.000113,0.000005,0.000008
std,4.602356,21.571763,1.301890,0.003878,0.006405,0.015988,0.003414,0.001423,0.018508,0.009232,0.004868,0.011267,0.007108,0.009106,0.038651,0.008121,0.009100,0.004625,0.016822,0.020403,0.005427,0.012611,0.007639,0.006625,0.006826,0.003652,0.006827,0.009769,0.004391,0.004827,0.007824,0.003126,0.005210,0.015672,0.01113,0.015889,0.010738,0.005338,0.005289,0.006782,0.026252,0.051697,0.003965,0.008868,0.000894,0.001462
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,5.000000,3.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,8.000000,4.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,14.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,262.000000,1203.000000,21.000000,0.705711,1.000000,1.000000,0.629027,0.262260,1.000000,0.715978,0.717017,1.000000,0.851152,1.000000,1.000000,1.000000,1.000000,0.687875,0.885539,1.000000,1.000000,1.000000,1.000000,0.769363,0.641044,0.468604,0.838829,0.728375,0.809081,0.510297,1.000000,0.576001,0.678525,1.000000,1.00000,1.000000,1.000000,0.779020,0.924854,1.000000,1.000000,1.000000,0.530717,1.000000,0.164654,0.269358


#### Análisis de sentimientos

Se intentó realizar un análisis de sentimientos, pero las alterniativas de traducción y la naturaleza del texto hizo bastante complicado lograr el objetivo, de modo que se omitió, pero se realizarón los ejercicios solicitados en la práctica.

### Parte 2 - Ingeniería continuas

**Objetivo**

Predecir el valor al cierre de las acciones de Facebook de la siguiente semana, utilizando técnicas de ingeniería de variables que nos permita construir una TAD para generar columnas que aporten valor a nuestro objetivo.

#### Librerias

In [7]:
from datetime import datetime
#import emoji
import jellyfish as jf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import unicodedata
import re

#### Dataset

In [13]:
fb = pd.read_csv('/work/files/workspace/practica_2/data/[M1] - FB stocks.csv')

In [10]:
fb.head()

,Fecha,Cierre,Apertura,Máximo,Mínimo,Quarter,Weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Fecha     1415 non-null   object 
 1   Cierre    1415 non-null   float64
 2   Apertura  1415 non-null   float64
 3   Máximo    1415 non-null   float64
 4   Mínimo    1415 non-null   float64
 5   Quarter   1415 non-null   int64  
 6   Weekday   1415 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 77.5+ KB


Modificamos la variable de fecha a un tipo datetime para su posterior manipulación

In [ ]:
fb['Fecha'] = pd.to_datetime(fb['Fecha'])

In [ ]:
fb.shape

(1415, 7)

#### Etiquetado de variables

**Antes del etiquetado de variables realizamos modificaciones en el nombre de las columnas**

In [ ]:
fb = fb.rename(columns=str.lower)
fb = fb.rename(columns={'máximo': 'maximo', 'mínimo':'minimo'})
fb.head()

,fecha,cierre,apertura,maximo,minimo,quarter,weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


**Realizamos el etiquetado de las variables**

Los prefijos que utilizaremos para los tipos de variables se describen de la siguiente manera:

**v_**: Variables categoricas.  
**c_**: Variables numericas (discretas y continuas).   
**d_**: Variables tipo fecha.  
**t_**: Variables de texto (comentarios, descripciones, url, etc.).  
**g_**: Variables geograficas.  


In [ ]:
v_feats=['quarter', 'weekday']
c_feats=['cierre','apertura',"maximo", "minimo"]
d_feats=['fecha']

v_feats_new=["v_"+x for x in v_feats]
c_feats_new=["c_"+x for x in c_feats]
d_feats_new=["d_"+x for x in d_feats]

fb.rename(columns=dict(zip(v_feats,v_feats_new)),inplace=True)
fb.rename(columns=dict(zip(c_feats,c_feats_new)),inplace=True)
fb.rename(columns=dict(zip(d_feats,d_feats_new)),inplace=True)

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1415 non-null   datetime64[ns]
 1   c_cierre    1415 non-null   float64       
 2   c_apertura  1415 non-null   float64       
 3   c_maximo    1415 non-null   float64       
 4   c_minimo    1415 non-null   float64       
 5   v_quarter   1415 non-null   int64         
 6   v_weekday   1415 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 77.5+ KB


In [ ]:
fb.shape

(1415, 7)

#### Calidad de datos

##### Duplicados

In [ ]:
#Total de registros duplicados de forma general
fb.duplicated().sum()

0

In [ ]:
#Total de registros duplicados por la variable de fecha
fb.duplicated(subset = 'd_fecha').sum()

0

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1415 non-null   datetime64[ns]
 1   c_cierre    1415 non-null   float64       
 2   c_apertura  1415 non-null   float64       
 3   c_maximo    1415 non-null   float64       
 4   c_minimo    1415 non-null   float64       
 5   v_quarter   1415 non-null   int64         
 6   v_weekday   1415 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 77.5+ KB


In [ ]:
fb.shape

(1415, 7)

##### Completitud

In [ ]:
def completitud(df):
    comple = pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace = True)
    comple = comple.rename(columns = {'index': 'columna', 0: 'nulos'})
    comple['completitud'] = (1-comple['nulos']/df.shape[0])*100
    comple = comple.sort_values(by = 'completitud', ascending = True)
    comple.reset_index(drop = True, inplace = True)
    return comple

completitud(fb)

,columna,nulos,completitud
0,d_fecha,0,100.0
1,c_cierre,0,100.0
2,c_apertura,0,100.0
3,c_maximo,0,100.0
4,c_minimo,0,100.0
5,v_quarter,0,100.0
6,v_weekday,0,100.0


In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,Friday
1,2012-01-08,20.88,21.50,21.58,20.84,1,Sunday
2,2012-01-10,21.99,22.08,22.59,21.73,1,Tuesday
3,2012-01-11,21.21,21.08,21.44,21.01,1,Wednesday
4,2012-02-07,30.77,31.25,31.73,30.55,1,Tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1415 non-null   datetime64[ns]
 1   c_cierre    1415 non-null   float64       
 2   c_apertura  1415 non-null   float64       
 3   c_maximo    1415 non-null   float64       
 4   c_minimo    1415 non-null   float64       
 5   v_quarter   1415 non-null   int64         
 6   v_weekday   1415 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 77.5+ KB


In [ ]:
fb.shape

(1415, 7)

##### Consistencia y precisión

###### Variables categoricas

In [ ]:
#Revisamos las variables categoricas
for i in fb.filter(like = 'v_'):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print('\n')

v_quarter


array(['1', '2', '3', '4'], dtype=object)



v_weekday


array(['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'], dtype=object)

**v_weekday**

Nos aseguramos de tomar los días correctos

In [ ]:
fb['v_weekday'] = fb['d_fecha'].map(lambda x: x.strftime('%A'))

Debido a que la naturaleza de los datos corresponden al precio de las acciones de **Facebook**, podemos inferir en que el los datos podrían ser inconsistentes al presentar datos en días de fin de semana, pues los **mercados financieros** solo funcionan y fijan precios de sus instrumentos de renta variable en los días de lunes a viernes.

En este sentido es viable eliminar estos registros y quedarnos con los que corresponden a los registros que deberían ser verídicos.

In [ ]:
#Modificaciones en la varible de días de la semana
fb['v_weekday'] = fb['v_weekday'].str.lower()
fb = fb.astype(dtype = {'v_weekday': 'string'})

#Eliminamos los datos fin de semana
fb = fb.drop(fb[fb['v_weekday'] == 'saturday'].index)
fb = fb.drop(fb[fb['v_weekday'] == 'sunday'].index)
fb.reset_index(drop = True, inplace = True)

In [ ]:
fb['v_weekday'].describe()

count       1265
unique         5
top       friday
freq         262
Name: v_weekday, dtype: object

In [ ]:
fb['v_weekday'].value_counts()

friday       262
wednesday    258
tuesday      256
thursday     250
monday       239
Name: v_weekday, dtype: Int64

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,friday
1,2012-01-10,21.99,22.08,22.59,21.73,1,tuesday
2,2012-01-11,21.21,21.08,21.44,21.01,1,wednesday
3,2012-02-07,30.77,31.25,31.73,30.55,1,tuesday
4,2012-02-08,20.04,20.77,20.84,19.82,1,wednesday


Ahora que hemos quitado los días correspondientes a los fines de semana, podemos observar que los días no son continuos y que nos falta información que podría ser de ayuda en futuros modelos, de modo que más adelante, el término de la revisión de calidad de datos haremos modificaciones para duplicar valores del periodo anterior para los casos que no tenemos

In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1265 non-null   datetime64[ns]
 1   c_cierre    1265 non-null   float64       
 2   c_apertura  1265 non-null   float64       
 3   c_maximo    1265 non-null   float64       
 4   c_minimo    1265 non-null   float64       
 5   v_quarter   1265 non-null   int64         
 6   v_weekday   1265 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 69.3 KB


In [ ]:
fb.shape

(1265, 7)

**v_quarter**

In [ ]:
fb['v_quarter'].value_counts()

3    340
4    333
2    310
1    282
Name: v_quarter, dtype: int64

Revisamos nuevamente

In [ ]:
#Revisamos de forma general las variables categoricas
for i in fb.filter(like="v_"):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print("\n")

v_quarter


array(['1', '2', '3', '4'], dtype=object)



v_weekday


array(['friday', 'monday', 'thursday', 'tuesday', 'wednesday'],
      dtype=object)

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,friday
1,2012-01-10,21.99,22.08,22.59,21.73,1,tuesday
2,2012-01-11,21.21,21.08,21.44,21.01,1,wednesday
3,2012-02-07,30.77,31.25,31.73,30.55,1,tuesday
4,2012-02-08,20.04,20.77,20.84,19.82,1,wednesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1265 non-null   datetime64[ns]
 1   c_cierre    1265 non-null   float64       
 2   c_apertura  1265 non-null   float64       
 3   c_maximo    1265 non-null   float64       
 4   c_minimo    1265 non-null   float64       
 5   v_quarter   1265 non-null   int64         
 6   v_weekday   1265 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 69.3 KB


In [ ]:
fb.shape

(1265, 7)

###### Variables númericas

In [ ]:
#Revisamos las variables continuas
for i in fb.filter(like= 'c_'):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print("\n")

c_cierre


array(['101.61', '101.91', '101.97', ..., '99.54', '99.67', '99.75'],
      dtype=object)



c_apertura


array(['100.41', '100.5', '100.58', ..., '99.0', '99.6', '99.88'],
      dtype=object)



c_maximo


array(['100.5', '100.58', '101.43', ..., '99.59', '99.73', '99.96'],
      dtype=object)



c_minimo


array(['100.16', '100.24', '100.4', ..., '97.55', '98.88', '99.75'],
      dtype=object)

In [ ]:
fb.describe()

,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter
count,1265.000000,1265.000000,1265.000000,1265.000000,1265.000000
mean,85.359779,85.367478,86.222379,84.431787,2.572332
std,45.661868,45.634000,45.830568,45.396818,1.103659
min,17.730000,18.080000,18.270000,17.550000,1.000000
25%,47.010000,46.990000,47.550000,46.260000,2.000000
50%,79.600000,79.540000,80.380000,78.630000,3.000000
75%,118.810000,118.920000,119.610000,117.670000,4.000000
max,183.030000,183.510000,184.250000,182.090000,4.000000


In [ ]:
#Revisamos si las variables continuas tienen datos no correspodientes a su naturaleza
for i in fb.filter(like = 'c_'):
    print(i)
    values = fb[fb[i].map(lambda x: sum([y.isalpha() for y in str(x)])>0)]
    values
    print("\n")

c_cierre


c_apertura


c_maximo


c_minimo




In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-01-06,27.72,28.89,29.15,27.39,1,friday
1,2012-01-10,21.99,22.08,22.59,21.73,1,tuesday
2,2012-01-11,21.21,21.08,21.44,21.01,1,wednesday
3,2012-02-07,30.77,31.25,31.73,30.55,1,tuesday
4,2012-02-08,20.04,20.77,20.84,19.82,1,wednesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265 entries, 0 to 1264
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1265 non-null   datetime64[ns]
 1   c_cierre    1265 non-null   float64       
 2   c_apertura  1265 non-null   float64       
 3   c_maximo    1265 non-null   float64       
 4   c_minimo    1265 non-null   float64       
 5   v_quarter   1265 non-null   int64         
 6   v_weekday   1265 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 69.3 KB


In [ ]:
fb.shape

(1265, 7)

###### Variables de fecha

In [ ]:
##Revisamos las variables fecha
for i in fb.filter(like="d_"):
    print(i)
    values = fb[i].astype(str).unique()
    values.sort()
    display(values)
    print("\n")

d_fecha


array(['2012-01-06', '2012-01-10', '2012-01-11', ..., '2017-12-27',
       '2017-12-28', '2017-12-29'], dtype=object)

In [ ]:
fb['d_fecha'].value_counts()

2012-01-06    1
2016-02-08    1
2016-02-19    1
2016-02-18    1
2016-02-17    1
             ..
2014-04-02    1
2014-03-31    1
2014-03-28    1
2014-03-27    1
2017-12-29    1
Name: d_fecha, Length: 1265, dtype: int64

In [ ]:
fb['d_fecha'].describe()

count                    1265
unique                   1265
top       2012-01-06 00:00:00
freq                        1
first     2012-01-06 00:00:00
last      2017-12-29 00:00:00
Name: d_fecha, dtype: object

Revisamos la frecuencia de registros mediante la fecha y podemos observar que hay una gran disparidad entre algunos registros, incluso en diferencias de registro de casi un mes

In [ ]:
fb["d_fecha"].diff().describe()

count                         1264
mean     1 days 17:28:06.075949367
std      1 days 17:58:11.332996044
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                2 days 00:00:00
max               27 days 00:00:00
Name: d_fecha, dtype: object

Agregamos el número de semana para poder encontrar las mayores distancias entre registros. Esto lo hacemos revisando cada año con ayuda de una nueva columna que representa el número de semana.

In [ ]:
fb['v_week_number'] = fb["d_fecha"].dt.week

Como se observa a continuación, el año 2012 es el peor registrado, pues de las 52 semanas que hubo en este año, solo se tienen registros de 40, sin considerar faltantes por semana.

Las semanas faltantes:

**11** se encuentran entre la semana **1 a 19**  
**1** se encuentra entre la 39 y 41

In [ ]:
anio = [2012, 2013, 2014, 2015, 2016, 2017]
for i in anio:
    print(f'Año: {i}')
    values = fb[fb['d_fecha'].map(lambda x: x.year == i)].reset_index(drop = True)
    values = values['v_week_number'].unique()
    print(f'Semanas registradas: {len(values)}')
    values.sort()
    display(values)
    print("\n")

Año: 2012
Semanas registradas: 40


array([ 1,  2,  6, 10, 11, 14, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52])



Año: 2013
Semanas registradas: 52


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])



Año: 2014
Semanas registradas: 52


...

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53])



Año: 2016
Semanas registradas: 52


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])



Año: 2017
Semanas registradas: 52


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])

Una opción es eliminar todos los datos del año 2012 donde no existe una regularidad en los registros, es decir, hasta la semana 19, pero también es importante ver no solo cuantas semanas hubo registros, sino también si esas semanas fueron completas o con un porcentaje aceptable.

Aqui definimos una regla:

Si suponemos que 1 semana entera de registro de los precios de las acciones de Facebook son de 5 registros, es decir, los correspondientes a cada día de Lunes a Viernes y un mes tiene 4 semanas, podemos decir que el número de registros en un mes es de 20, de modo que si tenemos un registro perdido por semana estamos diciendo que podemos aceptar la perdida de 5 registros al mes, es decir, tener 15 registros mensuales, lo que prespresenta un 75%, de modo que los meses que representen un porcentaje menor a este, no serán considerados.

Con base en lo anterior decidimos que registros eliminaremos para cada año si así se requiere.

Revisión año **2012**

In [ ]:
def faltante(df):
    reg = pd.DataFrame(fb['d_fecha'].map(lambda x: x.month)[fb["d_fecha"].map(lambda x: x.year == 2012)]
                       .value_counts())
    reg.reset_index(inplace=True)
    reg=reg.rename(columns={"index":"mes",'d_fecha':"registros_totales"})
    reg["proporcion"]=(reg["registros_totales"]/20)*100
    reg=reg.sort_values(by="mes",ascending=True)
    reg.reset_index(drop=True,inplace=True)
    return reg

faltante(fb)

,mes,registros_totales,proporcion
0,1,3,15.0
1,2,3,15.0
2,3,3,15.0
3,4,4,20.0
4,5,13,65.0
5,6,18,90.0
6,7,17,85.0
7,8,18,90.0
8,9,15,75.0
9,10,15,75.0


Obtenemos el porcentaje que representan estos elementos sobre el total, de ser bajo procedemos a eliminar los registros correspondientes a los meses de enero a mayo, ya que contienen < 75% de los datos mensuales.

In [ ]:
(fb['d_fecha'].map(lambda x: x.month)[(fb['d_fecha'].map(lambda x: x.month <= 5)) & 
                                      (fb['d_fecha'].map(lambda x: x.year == 2012))].value_counts().sum()
 /fb.shape[0]) * 100

2.0553359683794468

Dado que el porcentaje de datos perdidos si consideramos eliminar la ventana de registros con una mayor cantidad de (meses enero a mayo del año 2012) **missings**, entonces eliminamos.

In [ ]:
fb = fb[fb['d_fecha'] >= '2012-06-01']
fb = fb.reset_index(drop=True)
fb = fb.drop('v_week_number', axis=1)
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,friday
3,2012-06-11,21.17,21.24,21.37,20.99,2,monday
4,2012-06-12,26.97,27.68,27.75,26.82,2,tuesday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239 entries, 0 to 1238
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1239 non-null   datetime64[ns]
 1   c_cierre    1239 non-null   float64       
 2   c_apertura  1239 non-null   float64       
 3   c_maximo    1239 non-null   float64       
 4   c_minimo    1239 non-null   float64       
 5   v_quarter   1239 non-null   int64         
 6   v_weekday   1239 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 67.9 KB


In [ ]:
fb['d_fecha'].describe()

count                    1239
unique                   1239
top       2012-06-06 00:00:00
freq                        1
first     2012-06-06 00:00:00
last      2017-12-29 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb["d_fecha"].diff().describe()

count                         1238
mean     1 days 15:23:33.247172859
std      1 days 02:10:10.686775038
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                2 days 00:00:00
max               10 days 00:00:00
Name: d_fecha, dtype: object

In [ ]:
df = pd.DataFrame(pd.date_range(start='2012-06-06', end='2017-12-29'), columns=['d_fecha'])
df.head()

,d_fecha
0,2012-06-06
1,2012-06-07
2,2012-06-08
3,2012-06-09
4,2012-06-10


In [ ]:
fb = df.merge(fb, on='d_fecha', how='left')
fb

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2.0,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2.0,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2.0,friday
3,2012-06-09,NaN,NaN,NaN,NaN,NaN,<NA>
4,2012-06-10,NaN,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...
2028,2017-12-25,NaN,NaN,NaN,NaN,NaN,<NA>
2029,2017-12-26,175.99,176.63,177.00,174.67,4.0,tuesday
2030,2017-12-27,177.62,176.55,178.44,176.26,4.0,wednesday
2031,2017-12-28,177.92,177.95,178.94,177.68,4.0,thursday


In [ ]:
fb = fb.fillna(method='ffill')

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2.0,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2.0,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2.0,friday
3,2012-06-09,21.92,21.39,22.15,21.30,2.0,friday
4,2012-06-10,21.92,21.39,22.15,21.30,2.0,friday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2033 entries, 0 to 2032
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     2033 non-null   datetime64[ns]
 1   c_cierre    2033 non-null   float64       
 2   c_apertura  2033 non-null   float64       
 3   c_maximo    2033 non-null   float64       
 4   c_minimo    2033 non-null   float64       
 5   v_quarter   2033 non-null   float64       
 6   v_weekday   2033 non-null   string        
dtypes: datetime64[ns](1), float64(5), string(1)
memory usage: 191.6 KB


In [ ]:
fb["v_weekday"]=fb["d_fecha"].map(lambda x:x.strftime('%A'))
fb = fb.astype(dtype={'v_weekday': 'string'})
fb = fb.astype(dtype={'v_quarter': 'int'})

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,Wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,Thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,Friday
3,2012-06-09,21.92,21.39,22.15,21.30,2,Saturday
4,2012-06-10,21.92,21.39,22.15,21.30,2,Sunday


In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2033 entries, 0 to 2032
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     2033 non-null   datetime64[ns]
 1   c_cierre    2033 non-null   float64       
 2   c_apertura  2033 non-null   float64       
 3   c_maximo    2033 non-null   float64       
 4   c_minimo    2033 non-null   float64       
 5   v_quarter   2033 non-null   int64         
 6   v_weekday   2033 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 191.6 KB


In [ ]:
fb['v_weekday'] = fb['v_weekday'].str.lower()

In [ ]:
fb = fb.drop(fb[fb['v_weekday'] == 'saturday'].index)
fb = fb.drop(fb[fb['v_weekday'] == 'sunday'].index)
fb.reset_index(drop=True,inplace=True)

In [ ]:
fb.head()

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,friday
3,2012-06-11,21.17,21.24,21.37,20.99,2,monday
4,2012-06-12,26.97,27.68,27.75,26.82,2,tuesday


In [ ]:
fb['d_fecha'].describe()

count                    1453
unique                   1453
top       2012-06-06 00:00:00
freq                        1
first     2012-06-06 00:00:00
last      2017-12-29 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb["d_fecha"].diff().describe()

count                         1452
mean     1 days 09:35:12.396694214
std      0 days 19:11:48.073539184
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                1 days 00:00:00
max                3 days 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb.shape

(1453, 7)

Listo, hemos limpiado, ordenado y acompletado los registros de los precios de las acciones de Facebook, ahora tenemos registros diarios de los días entre semana.

#### Orden

Verificamos el orden por fecha

In [ ]:
fb["d_fecha"]=pd.to_datetime(fb["d_fecha"])
fb=fb.sort_values(by=["d_fecha"])
fb.reset_index(drop=True,inplace=True)

In [ ]:
fb.head(100)

,d_fecha,c_cierre,c_apertura,c_maximo,c_minimo,v_quarter,v_weekday
0,2012-06-06,26.81,26.07,27.17,25.52,2,wednesday
1,2012-06-07,31.73,31.44,31.90,31.26,2,thursday
2,2012-06-08,21.92,21.39,22.15,21.30,2,friday
3,2012-06-11,21.17,21.24,21.37,20.99,2,monday
4,2012-06-12,26.97,27.68,27.75,26.82,2,tuesday
...,...,...,...,...,...,...,...
95,2012-10-17,19.88,19.50,20.48,19.37,4,wednesday
96,2012-10-18,18.98,19.70,19.79,18.89,4,thursday
97,2012-10-19,19.00,19.00,19.06,18.80,4,friday
98,2012-10-22,19.32,19.20,19.43,19.05,4,monday


#### ¿Cada cuantos días se recopiló la información?

Nuestros datos ahora se presentan diarios de lunes a viernes.

In [ ]:
fb["d_fecha"].diff().describe()

count                         1452
mean     1 days 09:35:12.396694214
std      0 days 19:11:48.073539184
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                1 days 00:00:00
max                3 days 00:00:00
Name: d_fecha, dtype: object

In [ ]:
fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453 entries, 0 to 1452
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   d_fecha     1453 non-null   datetime64[ns]
 1   c_cierre    1453 non-null   float64       
 2   c_apertura  1453 non-null   float64       
 3   c_maximo    1453 non-null   float64       
 4   c_minimo    1453 non-null   float64       
 5   v_quarter   1453 non-null   int64         
 6   v_weekday   1453 non-null   string        
dtypes: datetime64[ns](1), float64(4), int64(1), string(1)
memory usage: 79.6 KB


#### Ingeniería de variables